In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from utils import *
from model.data_loader import *
from model.unet_baseline import unet
from model.unet_ternaus import ternausNet16
from model.unet_ternaus_tweaked import ternausNet16_tweaked
from model.unet_pix2pix import unet_pix2pix

In [3]:
train_pth = 'data/membrane/train'
test_pth = 'data/membrane/test'
results_pth = 'results/'
pretrained_pth = 'pretrained/'

# Baseline

Train the original unet model on the full 512px dataset until overfitting.

In [5]:
img_sz = (512, 512, 1)
batch_sz = 2

# Models
models = [
    ['unet_baseline', unet, dict(input_size=img_sz, transpose=True)]
]

# Data loaders
train_loader = loader(train_pth, input_generator_train, target_generator_train, 
                      batch_sz=batch_sz, img_sz=img_sz[:2])
test_loader = loader(test_pth, input_generator_test, target_generator_test, 
                     batch_sz=batch_sz, img_sz=img_sz[:2])

In [11]:
test_title = '512px_500steps_10epochs'

training_params = dict(
    train_steps=500, 
    val_steps=100, 
    epochs=10, 
    iterations=1, 
    lr=1e-4
)

for model in models:
    print(f'\nTESTING MODEL: {model[0]}')
    save_pth = f'{pretrained_pth}{model[0]}_{test_title}.h5'
    results = test_model(model[1], train_loader, test_loader, **training_params, 
                         model_params=model[2], save_pth=save_pth)
    results_df = hists2df(results)
    results_df.to_csv(f'{results_pth}{model[0]}_{test_title}.csv')


TESTING MODEL: unet_baseline
Epoch 1/10
500/500 [==============================] - 259s 519ms/step - loss: 0.5889 - acc: 0.8872 - val_loss: 0.4561 - val_acc: 0.9299
Epoch 2/10
500/500 [==============================] - 257s 515ms/step - loss: 0.4870 - acc: 0.9201 - val_loss: 0.4700 - val_acc: 0.9347
Epoch 3/10
500/500 [==============================] - 257s 514ms/step - loss: 0.4551 - acc: 0.9270 - val_loss: 0.4470 - val_acc: 0.9349
Epoch 4/10
500/500 [==============================] - 257s 514ms/step - loss: 0.4327 - acc: 0.9318 - val_loss: 0.4386 - val_acc: 0.9376
Epoch 5/10
499/500 [============================>.] - ETA: 0s - loss: 0.4139 - acc: 0.9358
Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
500/500 [==============================] - 255s 511ms/step - loss: 0.4139 - acc: 0.9358 - val_loss: 0.4374 - val_acc: 0.9367
Epoch 6/10
499/500 [============================>.] - ETA: 0s - loss: 0.4003 - acc: 0.9396
Epoch 00006: ReduceLROnPlateau reducing

Best Validation Accuracy: 0.9376

# Full Test: 20 epochs of 250 training steps

In [4]:
img_sz = (256, 256, 1)
batch_sz = 2

# Models
models = [
#     ['unet_baseline',           unet,                 dict(input_size=img_sz, transpose=True)],
#     ['unet_baseline_upsampled', unet,                 dict(input_size=img_sz, transpose=False)],
#     ['unet_ternaus_nopre',      ternausNet16_tweaked, dict(input_size=img_sz, dropout=False, batch_norm=False, pretrained=False)],
#     ['unet_ternaus',            ternausNet16,         dict(input_size=img_sz)],
#     ['unet_ternaus_drop',       ternausNet16_tweaked, dict(input_size=img_sz, dropout=True,  batch_norm=False, pretrained=True)],
#     ['unet_ternaus_bn',         ternausNet16_tweaked, dict(input_size=img_sz, dropout=False, batch_norm=True,  pretrained=True)],
#     ['unet_ternaus_dropbn',     ternausNet16_tweaked, dict(input_size=img_sz, dropout=True,  batch_norm=True,  pretrained=True)],
    ['unet_pix2pix',            unet_pix2pix, dict(input_size=img_sz)]
]

# Data loaders
train_loader = loader(train_pth, input_generator_train, target_generator_train, batch_sz=batch_sz, img_sz=img_sz[:2])
test_loader = loader(test_pth, input_generator_test, target_generator_test, batch_sz=batch_sz, img_sz=img_sz[:2])

In [5]:
test_title = '256px_250steps_20epochs'

training_params = dict(
    train_steps=250, 
    val_steps=100, 
    epochs=20, 
    iterations=5, 
    lr=1e-4
)

for model in models:
    print(f'\nTESTING MODEL: {model[0]}')
    save_pth = f'{pretrained_pth}{model[0]}_{test_title}.h5'
    results = test_model(model[1], train_loader, test_loader, **training_params, 
                         model_params=model[2], save_pth=save_pth)
    results_df = hists2df(results)
    results_df.to_csv(f'{results_pth}{model[0]}_{test_title}.csv')


TESTING MODEL: unet_pix2pix
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/20
249/250 [============================>.] - ETA: 0s - loss: 4.1364 - acc: 0.2600Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
250/250 [==============================] - 42s 169ms/step - loss: 4.1305 - acc: 0.2605 - val_loss: 2.6649 - val_acc: 0.3605
Epoch 2/20
250/250 [==============================] - 37s 147ms/step - loss: 1.3803 - acc: 0.6194 - val_loss: 1.1007 - val_acc: 0.7706
Epoch 3/20
250/250 [==============================] - 37s 146ms/step - loss: 0.9906 - acc: 0.7967 - val_loss: 0.8961 - val_acc: 0.8289
Epoch 4/20
250/250 [==============================] - 37s 147ms/step - loss: 0.8399 - acc: 0

250/250 [==============================] - 35s 140ms/step - loss: 0.6990 - acc: 0.8403 - val_loss: 0.6606 - val_acc: 0.8650
Epoch 8/20
250/250 [==============================] - 35s 141ms/step - loss: 0.6680 - acc: 0.8491 - val_loss: 0.6585 - val_acc: 0.8715
Epoch 9/20
250/250 [==============================] - 35s 140ms/step - loss: 0.6465 - acc: 0.8557 - val_loss: 0.6218 - val_acc: 0.8758
Epoch 10/20
100/100 [==============================] - 3s 28ms/step - loss: 0.6285 - acc: 0.8750

Epoch 00010: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
250/250 [==============================] - 33s 130ms/step - loss: 0.6278 - acc: 0.8612 - val_loss: 0.6285 - val_acc: 0.8750
Epoch 11/20
250/250 [==============================] - 35s 140ms/step - loss: 0.6150 - acc: 0.8651 - val_loss: 0.6075 - val_acc: 0.8802
Epoch 12/20
250/250 [==============================] - 35s 140ms/step - loss: 0.6136 - acc: 0.8656 - val_loss: 0.6071 - val_acc: 0.8817
Epoch 13/20
100/100 [===========

Epoch 15/20
250/250 [==============================] - 36s 145ms/step - loss: 0.5343 - acc: 0.8873 - val_loss: 0.5203 - val_acc: 0.8974
Epoch 16/20
250/250 [==============================] - 36s 143ms/step - loss: 0.5334 - acc: 0.8873 - val_loss: 0.4996 - val_acc: 0.8992
Epoch 17/20
250/250 [==============================] - 36s 143ms/step - loss: 0.5315 - acc: 0.8876 - val_loss: 0.5021 - val_acc: 0.9007
Epoch 18/20
100/100 [==============================] - 3s 34ms/step - loss: 0.5153 - acc: 0.8982

Epoch 00018: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
250/250 [==============================] - 33s 134ms/step - loss: 0.5297 - acc: 0.8879 - val_loss: 0.5153 - val_acc: 0.8982
Epoch 19/20
100/100 [==============================] - 3s 34ms/step - loss: 0.4975 - acc: 0.9005

Epoch 00019: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
250/250 [==============================] - 33s 134ms/step - loss: 0.5283 - acc: 0.8882 - val_loss: 0.4975 - val_

TESTING MODEL: unet_baseline 
train_steps=1000
epochs=5

new augmentation

loss: 0.4721 - acc: 0.8949 - val_loss: 0.3291 - val_acc: 0.9325
loss: 0.3288 - acc: 0.9284 - val_loss: 0.3316 - val_acc: 0.9346
loss: 0.2828 - acc: 0.9381 - val_loss: 0.3206 - val_acc: 0.9349
loss: 0.2543 - acc: 0.9443 - val_loss: 0.3570 - val_acc: 0.9299  <--overfitting
loss: 0.2304 - acc: 0.9496 - val_loss: 0.3599 - val_acc: 0.9341

512px 2000 steps
loss: 1.1966 - acc: 0.9010 - val_loss: 1.0991 - val_acc: 0.9367
loss: 1.0032 - acc: 0.9439 - val_loss: 1.0317 - val_acc: 0.9312  <--overfitting

old augmentation

,val_loss,val_acc,loss,acc,experiment,epoch
0,0.3938294525444508,0.9204465866088867,0.5577377219498157,0.8792923736572266,0.0,0.0
1,0.3559911660850048,0.9349861907958984,0.4401548975408077,0.9053520278930665,0.0,1.0
2,0.39109488159418104,0.9267212295532227,0.4238285449743271,0.9082808227539062,0.0,2.0    <--overfitting
3,0.4155013260245323,0.9168938446044922,0.42954228895902635,0.9082791290283203,0.0,3.0
4,0.36537827536463735,0.9267246627807617,0.39881821677088736,0.9130117721557617,0.0,4.0

5,0.49226998195052146,0.9182594680786133,0.6562534253895282,0.8670453491210938,1.0,0.0
6,0.4583278933167458,0.9273256683349609,0.554767794162035,0.8987517471313476,1.0,1.0
7,0.4612529504299164,0.9287994384765625,0.5254129691720009,0.904376106262207,1.0,2.0
8,0.4672836236655712,0.9274766540527344,0.5032809109687805,0.9079330139160157,1.0,3.0   <--overfitting
9,0.4568671178817749,0.922571907043457,0.5079058838784695,0.9074051818847656,1.0,4.0

10,1.3314954286813736,0.7966962432861329,1.3639041519165038,0.7678601226806641,2.0,0.0
11,1.282408196926117,0.7967258834838867,1.3139923479557036,0.7705478591918945,2.0,1.0
12,1.2392406225204469,0.7965563201904297,1.273618043422699,0.7686567840576172,2.0,2.0
13,1.2005380243062973,0.7966357803344727,1.237180896639824,0.769509765625,2.0,3.0
14,1.166281766295433,0.7965296936035157,1.2050516810417176,0.7705629653930665,2.0,4.0

15,0.47512905225157737,0.9222694778442383,0.6407421391904354,0.8729847259521485,3.0,0.0
16,0.4551257087290287,0.9326386260986328,0.5516852941811085,0.8992034530639649,3.0,1.0
17,0.47955494463443754,0.9300589370727539,0.5159895688593388,0.9065925064086914,3.0,2.0 
18,0.4419742250442505,0.9331255340576172,0.5083408271372318,0.9076029663085937,3.0,3.0
19,0.4340150611102581,0.9320641326904296,0.4795567348897457,0.9116422729492187,3.0,4.0    <--overfitting

20,0.37043760359287264,0.9277394485473632,0.5708116674721241,0.8749972534179687,4.0,0.0
21,0.3530151304602623,0.9324607086181641,0.4412955792546272,0.9052258071899414,4.0,1.0
22,0.3365512517094612,0.935040283203125,0.43223093220591546,0.9074309005737304,4.0,2.0
23,0.3444555760920048,0.930106315612793,0.4199221305847168,0.9092439498901367,4.0,3.0   <--overfitting
24,0.34377639517188074,0.9305371475219727,0.40034309843182564,0.9129037475585937,4.0,4.0




TESTING MODEL: unet_ternaus
train_steps=2000
epochs=5

loss: 0.3033 - acc: 0.9342 - val_loss: 0.3997 - val_acc: 0.9302
loss: 0.1896 - acc: 0.9586 - val_loss: 0.4191 - val_acc: 0.9312
loss: 0.1561 - acc: 0.9660 - val_loss: 0.4229 - val_acc: 0.9336
loss: 0.1372 - acc: 0.9701 - val_loss: 0.4552 - val_acc: 0.9294  <--overfitting

# Fast test: 50 training steps

## Compare baseline implementation to more recent u-net implementations


TODO: update

It is observed that the final accuracy of both models varies in the range 80-96% when trained for 5 epochs with 2000 iterations. How can we quickly compare the performance of these two models?

Here's a quick comparison of models by training each for 50 training steps from scratch 30 times. We compare our baseline implementation to [zhixuhao's model](https://github.com/zhixuhao/unet) and find no significant difference.

In [6]:
test_title = '256px_50steps'

training_params = dict(
    train_steps=50, 
    val_steps=100, 
    epochs=1, 
    iterations=20, 
    lr=1e-4
)

for model in models:
    print(f'\nTESTING MODEL: {model[0]}')
    results = test_model(model[1], train_loader, test_loader, **training_params, 
                         model_params=model[2])
    results_df = hists2df(results)
    results_df.to_csv(f'{results_pth}{model[0]}_{test_title}.csv')


TESTING MODEL: unet_pix2pix
50/50 [==============================] - 13s 250ms/step - loss: 9.2893 - acc: 0.2209 - val_loss: 5.7268 - val_acc: 0.2031


# Data augmentation tuning

## data_loader_old

In [4]:
def fast_test(model_fn, iterations, steps_per_iter, epochs=1, lr=1e-4):
    hists = []
    for i in range(iterations):
        model = model_fn(input_size=(256, 256, 1))
        model.compile(optimizer = Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])
        history = model.fit_generator(train_gen, steps_per_epoch=steps_per_iter, epochs=1)
        hists.append(history.history)
#         r = {k: r[k][-1] for k in r}
#         df = df.append(r, ignore_index=True)
        backend.clear_session()
    return hists

In [5]:
# TODO: Implement test set loader

data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')

# why is batch size 2? paper says bs=1
train_gen = trainGenerator(2, train_pth, 'input', 'target', data_gen_args, save_to_dir=None)



In [6]:
hists_baseline_slow = fast_test(unet, 5, 1000, epochs=1, lr=1e-4)

Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
1000/1000 [==============================] - 161s 161ms/step - loss: 0.5742 - acc: 0.8896
Epoch 1/1
1000/1000 [==============================] - 159s 159ms/step - loss: 0.5651 - acc: 0.8921
Epoch 1/1
1000/1000 [==============================] - 158s 158ms/step - loss: 0.4570 - acc: 0.8995
Epoch 1/1


KeyboardInterrupt: 

## data_loader

In [25]:
img_sz = (256, 256, 1)
batch_sz = 2

# Models
models = [
    ['unet_baseline', unet, dict(input_size=img_sz)],
]

# Data loaders
train_loader = loader(train_pth, input_generator_train, target_generator_train, batch_sz=batch_sz, img_sz=img_sz[:2])
test_loader = loader(test_pth, input_generator_test, target_generator_test, batch_sz=batch_sz, img_sz=img_sz[:2])

In [26]:
training_params = dict(
    train_steps=1000, 
    val_steps=200, 
    epochs=1, 
    iterations=3, 
    lr=1e-4
)

for model in models:
    print(f'\nTESTING MODEL: {model[0]}')
    results = test_model(model[1], train_loader, test_loader, **training_params, model_params=model[2])


TESTING MODEL: unet_baseline
Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
 999/1000 [============================>.] - ETA: 0s - loss: 0.4675 - acc: 0.8973Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
1000/1000 [==============================] - 166s 166ms/step - loss: 0.4675 - acc: 0.8973 - val_loss: 0.3348 - val_acc: 0.9324
Epoch 1/1
1000/1000 [==============================] - 167s 167ms/step - loss: 0.4851 - acc: 0.8936 - val_loss: 0.3304 - val_acc: 0.9326
Epoch 1/1
1000/1000 [==============================] - 167s 167ms/step - loss: 1.2724 - acc: 0.8682 - val_loss: 1.1669 - val_acc: 0.9347


### Test A

input_generator_train = ImageDataGenerator(
     rotation_range=180,
     width_shift_range=0.2,
     height_shift_range=0.2,
     brightness_range=[0.5, 1.5],
     rescale=1./255,           #  rescale pixel vals 0-255 --> 0.0-1.0
     shear_range=0.2,
     zoom_range=[0.5, 1.0],
     horizontal_flip=True,
     fill_mode='reflect',
     data_format='channels_last',
     validation_split=0.0
 )

loss: 0.5713 - acc: 0.8766 - val_loss: 0.3788 - val_acc: 0.9237
loss: 0.6409 - acc: 0.8713 - val_loss: 0.4842 - val_acc: 0.9206
loss: 1.2721 - acc: 0.8673 - val_loss: 1.1907 - val_acc: 0.9294


### Test B

input_generator_train = ImageDataGenerator(
    rotation_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    #brightness_range=[0.5, 1.5],
    rescale=1./255,           #  rescale pixel vals 0-255 --> 0.0-1.0
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest', #nearest
    data_format='channels_last',
    validation_split=0.0
)

loss: 0.4986 - acc: 0.8909 - val_loss: 0.3177 - val_acc: 0.9342
loss: 1.3596 - acc: 0.7805 - val_loss: 1.3306 - val_acc: 0.7968
loss: 0.4908 - acc: 0.8891 - val_loss: 0.3676 - val_acc: 0.9292


### Test C

as B:
fill_mode='reflect'

loss: 0.5527 - acc: 0.8786 - val_loss: 0.3485 - val_acc: 0.9337
loss: 0.5636 - acc: 0.8933 - val_loss: 0.4527 - val_acc: 0.9340
loss: 0.4779 - acc: 0.8942 - val_loss: 0.3359 - val_acc: 0.9326


### Test D

as C:
rotation_range=0.0,

loss: 0.5775 - acc: 0.8873 - val_loss: 0.4613 - val_acc: 0.9319
loss: 1.2456 - acc: 0.8952 - val_loss: 1.1706 - val_acc: 0.9309
loss: 0.4818 - acc: 0.8946 - val_loss: 0.3495 - val_acc: 0.9314

### Test E

as C
brightness_range=[0.8, 1.2]

loss: 0.4811 - acc: 0.8946 - val_loss: 0.3346 - val_acc: 0.9327
loss: 0.4701 - acc: 0.8968 - val_loss: 0.4053 - val_acc: 0.9273
loss: 0.4712 - acc: 0.8971 - val_loss: 0.3999 - val_acc: 0.9275

### Test F

as C
rotation_range=2,
brightness_range=[0.8, 1.2]

loss: 0.4675 - acc: 0.8973 - val_loss: 0.3348 - val_acc: 0.9324
loss: 0.4851 - acc: 0.8936 - val_loss: 0.3304 - val_acc: 0.9326
loss: 1.2724 - acc: 0.8682 - val_loss: 1.1669 - val_acc: 0.9347